# Importing Libraries

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from sklearn import preprocessing, metrics
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
from sklearn.model_selection import GroupKFold, KFold, TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.layers import Input, Flatten, Concatenate, BatchNormalization, Embedding
from tensorflow.keras.losses import mse
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.compat.v1.keras.layers import CuDNNLSTM



In [0]:
w_size = 30 # LSTM window size
batch_size=512
epochs = 35
span_lst = [7, 30, 90] # moving avarage time windows

In [0]:
sales = pd.read_csv('/content/drive/My Drive/m5-forecasting-accuracy/sales_train_validation.csv')# read sales data
# categories are used for categorical model input
categories = sales[['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]# group categories
sales['id'] = sales['id'].apply(lambda x: x[:-11]) # in id remove '_validation' so we get products with state
ids = sales['id'].values

sales = sales.iloc[:, -500:].T.reset_index()  #taking last 500 days for training and transposing it
    
sales.columns = ['d'] + list(ids)   #combine days and items

In [0]:
calendar = pd.read_csv('/content/drive/My Drive/m5-forecasting-accuracy/calendar.csv') # reading calendar data

In [0]:
sales.d[0][2:]

'1414'

In [0]:
calendar.d[2:]

2          d_3
3          d_4
4          d_5
5          d_6
6          d_7
         ...  
1964    d_1965
1965    d_1966
1966    d_1967
1967    d_1968
1968    d_1969
Name: d, Length: 1967, dtype: object

In [0]:
# subsetting by starting date here d_1414                             #'1414'
calendar = calendar[calendar.d.apply(lambda x: int(x[2:])) >= int(sales.d[0][2:])]


# Price Dataset

* Transposing the long format into (weeks, items) dimension

In [0]:
price = pd.read_csv('/content/drive/My Drive/m5-forecasting-accuracy/sell_prices.csv') #reading price data

In [0]:
#get the prices of items from starting day i.e 1414 and corresponding wm_yr_wk 11445
price = price.loc[price.wm_yr_wk >= \
                    calendar[calendar.d == sales.d[0]]['wm_yr_wk'].values[0]]

In [0]:
price['id'] = price.apply(lambda x: x.item_id + '_' + x.store_id, axis=1) # combine item_id and store_id to get id as in sales
price = price.pivot(index='wm_yr_wk', columns='id', values='sell_price') # form a table with wm_yr_wk as rows and id as cols
price = calendar[['d','wm_yr_wk']].merge(price, how='inner', on=['wm_yr_wk']) # merge with calendar
price.drop('wm_yr_wk', axis=1, inplace=True)
price = price.loc[:, list(sales.columns)] #get the price for all items from starting day
calendar.drop(['date','wm_yr_wk', 'weekday', 'd'], axis=1, inplace=True)


# Preprocessing

*  Both sales and prices are log scaled and then standardized by global average and std.


In [0]:
sales_log = np.log(sales.iloc[:, 1:].values + 1)
sales_mean = np.mean(sales_log)
sales_std = np.std(sales_log)
sales.iloc[:, 1:] = (sales_log - sales_mean) / sales_std

price_log = np.log(price.iloc[:, 1:].values)
price_mean = np.mean(price_log)
price_std = np.std(price_log)
price.iloc[:, 1:] = (price_log - price_mean) / price_std

sales.fillna(0, inplace=True)
price.fillna(0, inplace=True)

# Label encoding Categorical features for embeddings.

In [0]:
cat_ft1 = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'] # from sales
cat_ft2 = ['wday','month', 'year', 'event_name_1', 'event_type_1',
           'event_name_2', 'event_type_2'] # from calendar

category_counts = {}
state_le = None

def LabelEncoding(df, cat_ft):
    
    for col in cat_ft:
        le = LabelEncoder()
        df.loc[:, col] = df[col].astype(str)
        df.loc[:, col] = le.fit_transform(df[col])
        category_counts[col] = len(list(le.classes_))

    return df

categories = LabelEncoding(categories, cat_ft1)
calendar = LabelEncoding(calendar, cat_ft2)

# SequenceGenerator for generating sequences similar to raw input

In [0]:
def moving_average(a, n): # calculating moving average
    
    if a.shape[0] >= n:
        ret = np.cumsum(a, axis=0) #compute the cumulative sum of array elements 
        ret[n:, :] = ret[n:, :] - ret[:-n, :]
        ret[:n-1, :] = np.zeros((n-1, ret.shape[1]))
        return ret / n
    else:
        return np.zeros((a.shape[0], a.shape[1]))
    
class SequenceGenerator:
    #calculate moving avg for 7 days in a window of 30 days with batch size of 32
    def __init__(self, inputs, spans=[7], window=30, batch_size=32, infer=False): #infer=prediction for now let's assign it to false
        self.sales = inputs[0]
        self.prices = inputs[1]
        self.categories = inputs[2]
        self.calendar = inputs[3]
        self.spans = spans
        self.window = window
        self.infer = infer
        self.num_items = self.sales.shape[1]
        
        if self.infer:                                                        
            self.batch_size = self.num_items                           # for training
            self.num_days = self.sales.shape[0] - self.window + 1
            self.steps_per_day = 1 
            self.steps = 1
        else:
            self.batch_size = batch_size
            self.num_days = self.sales.shape[0] - self.window          # for prediction
            self.steps_per_day = self.num_items // self.batch_size + 1
            self.steps = self.steps_per_day * self.num_days

    def generate(self):
        
        ## for prediction, it starts from the the last starting date (no slides in days)
        ## for training/validation, it starts from day 0
        start_day = self.num_days - 1 if self.infer else 0
            
        while True:            
            
            for day in range(start_day, self.num_days):
                    
                s = self.sales[day:day+self.window, :].reshape(1, self.window, -1) #num of sales for window
                p = self.prices[day:day+self.window, :]\
                    .reshape(1, self.window, -1)   #prices for window

                X = np.concatenate((s,p),axis=0)

                for span in self.spans:
                    
                    span_ = day if day < span else span

                    ma = moving_average(self.sales[day-span_:day+self.window, :] #calculate mov avg for given span in that window
                                        , n=span)[span_:, :]\
                        .reshape(1, self.window, -1)

                    X = np.concatenate((X, ma),axis=0)

                ## transposing (features, days, items) into (items, days, features)
                X = np.transpose(X, (2,1,0)) 

                if not self.infer: #while predicting first output is considered as input to next day
                    y = self.sales[day+self.window, :].reshape(-1,1) 
                    
                for i in range(self.steps_per_day):
                    
                    ## if the batch go over the maxium item number, 
                    ## the batch_size will be truncated
                    if (i+1)*self.batch_size > self.num_items:
                        end = self.num_items
                    else:
                        end = (i+1)*self.batch_size
                    
                    ## categories has (items, features) shape
                    ## only relevant item rows are fetched
                    cat = self.categories[i*self.batch_size:end, :]
                    state_id = cat[:, -1]
                    # reshaping into (features, items, 1)
                    cat = cat.T.reshape(cat.shape[1], cat.shape[0], 1)
                    
                    ## calender values are taken at prediction target date
                    calen = self.calendar[day+self.window,:7].reshape(1,-1)
                    calen = np.repeat(calen, end-i*self.batch_size, axis=0)
                    calen = calen.T.reshape(calen.shape[1], calen.shape[0], 1)
                    
                    ## snap values are taken at prediction target date
                    snap = self.calendar[day+self.window,7:].reshape(1,-1)
                    snap = np.repeat(snap, end-i*self.batch_size, axis=0)
                    # taking only relevant state's snap values for each row
                    snap = snap[np.arange(len(snap)), state_id].reshape(-1,1)
                    
                    if self.infer:
                        yield [X[i*self.batch_size: end]] + [j for j in cat]\
                                + [j for j in calen] + [snap]
                    else:
                        yield [X[i*self.batch_size: end]] + [j for j in cat] \
                              + [j for j in calen] + [snap],\
                              y[i*self.batch_size: end]

In [0]:
def define_model(lstm_w_size, lstm_n_fts): #for lstm cell modeling
    
    ## Categorical embedding
    cat_inputs = []
    for cat in cat_ft1+cat_ft2:
        cat_inputs.append(Input(shape=[1], name=cat))
        
    cat_embeddings = []
    for i, cat in enumerate(cat_ft1+cat_ft2):
        cat_embeddings.append(Embedding(category_counts[cat], 
                                        min(50, int(category_counts[cat]+1/ 2)), 
                                        name = cat + "_embed")(cat_inputs[i]))

    cat_output = Concatenate()([Flatten()(cat_emb) \
                                          for cat_emb in cat_embeddings])
    cat_output = Dropout(.7)(cat_output)
    
    # snap input
    snap_input = Input(shape=[1])

    ## LSTM
    lstm_input = Input(shape=(lstm_w_size, lstm_n_fts)) #input shape
    lstm_output = CuDNNLSTM(32)(lstm_input) #o/p shape is 32
    
    concat = Concatenate()([
        lstm_output,
        cat_output,
        snap_input
    ])
        
    dense_output = Dense(10,LeakyReLU(alpha=0.1))(concat)
    out = Dense(1)(dense_output)
    model = Model(inputs=[lstm_input] + cat_inputs + [snap_input],
                  outputs=out)
    model.compile(optimizer='rmsprop', loss='mse')
    
    return model

# Model Training - Cross Validation

In [0]:
def model_training(inputs, cv, w_size=30, batch_size=32, epochs=10,
                   early_stopping=10):

    val_scores=[]
    train_evals=[]
    valid_evals=[]
    best_epoch=[]

    for idx, (train_index, val_index) in enumerate(cv.split(inputs[0])):
        
        if idx >= 1: # skipping the first 2 fold to save run time

            #print("###### fold %d ######" % (idx+1))
            sales_train, sales_val = inputs[0][train_index, :],\
                                     inputs[0][val_index, :]
            prices_train, prices_val = inputs[1][train_index, :],\
                                       inputs[1][val_index, :]
            calendar_train, calendar_val = inputs[3][train_index, :],\
                                           inputs[3][val_index, :]
            inputs_train = [sales_train, prices_train, inputs[2], calendar_train]
            inputs_val = [sales_val, prices_val, inputs[2], calendar_val]

            train_gen = SequenceGenerator(inputs_train, spans=span_lst,
                                          window=w_size, batch_size=batch_size)
            val_gen = SequenceGenerator(inputs_val, spans=span_lst, window=w_size,
                                        batch_size=batch_size)

            model = define_model(w_size, 2+len(span_lst))
            early_stop = EarlyStopping(patience=early_stopping,
                                       verbose=True,
                                       restore_best_weights=True)

            hist = model.fit_generator(train_gen.generate(),
                      validation_data=val_gen.generate(),
                      epochs=epochs,
                      steps_per_epoch=train_gen.steps, 
                      validation_steps=val_gen.steps, 
                      callbacks=[early_stop],
                      verbose=0)

            val_scores.append(np.min(hist.history['val_loss']))
            train_evals.append(hist.history['loss'])
            valid_evals.append(hist.history['val_loss'])

            best_epoch.append(np.argmin(hist.history['val_loss']) + 1)
    
    print('### CV scores by fold ###')
    for i in range(2, cv.get_n_splits(sales)):
        print(f'fold {i+1}: {val_scores[i-2]:.4f} at epoch {best_epoch[i-2]}')
    print('CV mean score: {0:.4f}, std: {1:.4f}'\
          .format(np.mean(val_scores), np.std(val_scores)))
    
    return best_epoch

In [0]:
sales = sales.iloc[:,1:].values
price = price.iloc[:,1:].values
categories = categories.values
calendar = calendar.values
inputs = [sales, price, categories, calendar]


cv = TimeSeriesSplit(n_splits=4)
best_epoch = model_training(inputs, cv, w_size=w_size, 
                                batch_size=batch_size, 
                                epochs=epochs, early_stopping=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Restoring model weights from the end of the best epoch.
Epoch 00020: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00018: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
### CV scores by fold ###
fold 3: 0.4651 at epoch 15
fold 4: 0.4753 at epoch 13
CV mean score: 0.4757, std: 0.0088


# Model Training without CV split

In [0]:
%%time
train_gen = SequenceGenerator(inputs, spans=span_lst, window=w_size,
                              batch_size=batch_size)
model = define_model(w_size, 2+len(span_lst))
hist = model.fit_generator(train_gen.generate(),
                           epochs=best_epoch[-1],
                           steps_per_epoch=train_gen.steps,
                           verbose=0)


CPU times: user 1h 3min 28s, sys: 5min 33s, total: 1h 9min 2s
Wall time: 49min 35s


# Predictions for Submission

* Each prediction will be used for the model input for the next day prediction



In [0]:
# subsetting len-w_size-90: as we need the first 90 days 
# prior to the LSTM window to calculate 90 days moving avarage
sales_test = sales[sales.shape[0]-w_size-90:, :]
prices_test = price[sales.shape[0]-w_size-90:, :]
calendar_test = calendar[sales.shape[0]-w_size-90:, :]
test_inputs = [sales_test, prices_test, categories, calendar_test]

for i in range(28):
    
    test_gen = SequenceGenerator(test_inputs, spans=span_lst, 
                                 window=w_size, infer=True)
    test_iter = test_gen.generate()
    X = next(test_iter)
    y_pred = model.predict(X)

    # appending predicted sales to the input and shifting it by 1
    sales_test = np.append(sales_test, y_pred.reshape(1,-1), axis=0)[1:, :]
    prices_test = prices_test[1:, :]
    calendar_test = calendar_test[1:, :]
    test_inputs = [sales_test, prices_test, categories, calendar_test]

In [0]:
sales_test = np.exp((sales_test * sales_std) + sales_mean) - 1
sales_test = np.maximum(sales_test, np.zeros(sales_test.shape))

In [0]:
submission = pd.read_csv('/content/drive/My Drive/m5-forecasting-accuracy/sample_submission.csv')

submission.iloc[:len(submission)//2, 1:] = sales_test[-28:, :].T
    
submission.to_csv('/content/drive/My Drive/m5-forecasting-accuracy/submission.csv', index=False)
submission.head()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dept_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
cat_id (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
store_id (InputLayer)           [(None, 1)]          0                                            
____________________________________________________________________________________________